# 검색과 응답을 최적화하는 RAG 고도화 전략
- RAG 고도화는 관련 문서를 가져오는 Retrieval 파트와 응답을 생성하는 Generation 파트 모두에서 달성할 수 있습니다.  

<table>
    <thead>
        <tr>
            <th>구분</th>
            <th>주요 개선 요소</th>
            <th>개략적 개선 방향</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan="4">Retrieval<br />(문서 검색)</td>
            <td>청킹 전략<br />(전처리)</td>
            <td>문서를 효과적으로 분할하는 다양한 기법을 활용하여 검색에 최적화딘 형태로 변환함으로써 검색 품질과 처리 속도 향상</td>
        </tr>
        <tr>            
            <td>질의 변형<br />(쿼리 재작성)</td>
            <td>질문의 구체화, 가상의 답변 활용 등의 기법을 통해 사용자의 원래 질문을 검색에 효율적인 형태로 재구성</td>
        </tr>        
        <tr>            
            <td>검색 알고리즘<br />최적화</td>
            <td>의미 기반의 벡터 검색과 키워드 기반 검색, 하이브리드 검색 등의 방식을 활용하여 관련성 높은 문서를 더 효율적으로 검색</td>
        </tr>           
        <tr>            
            <td>문서 후처리<br />(리랭킹)</td>
            <td>1차 검색 결과와 질문의 연관성을 재평가하여 질문과 관련없는 문서 제거, 이를 통해 검색된 정보의 품질을 높이고 궁극적으로 더 신뢰할 수 있는 최종 프롬프트 제공</td>
        </tr>
        <tr>
            <td rowspan="2">Generation<br />(답변 생성)</td>
            <td>Self-RAG</td>
            <td>모델이 스스로 추가 검색 필요성을 판단하고 부족한 정보를 찾아 보완. 이를 바탕으로 더 깊이 있고 정확한 응답을 제공하여 사용자 경험 개선</td>
        </tr>                                
        <tr>            
            <td>모델 파인튜닝</td>
            <td>모델을 학습시켜 다양한 상황에 대한 대응 능력 강화. 이를 통해 검색된 정보를 더욱 효율적으로 활용하여 맥락에 맞는 적절한 응답 생성 가능</td>
        </tr>        
    </tbody>    
</table>

## 1. 청킹 전략
- 문서 전처리 단계는 RAG 시스템에서 실제 검색에 사용될 문서들을 가공하여, 검색과 생성에 최적화된 형태로 변환하는 과정입니다.
- 문서 분할은 긴 문서를 더 작고 관리하기 쉬운 단위로 나누는 과정입니다.
- 효과적인 분할 방식은 관련 정보의 정확도를 높이고, 문맥의 일관성을 유지하는 데 도움을 줍니다.
- 가장 기본적인 분할 방식은 문자 수 기반 분할입니다.
- 구현히 가장 간단하고 빠르지만, 의미적 구조를 고려하지 않기 때문에, 중요한 정보가 분할되어 검색시 누락되거나 문맥이 왜곡될수 있습니다.

- 재귀적 문자 텍스트 분할 방식은 문서의 구조를 고려하여 분할하지만, 의미를 고려하지 않고 기계적인 규칙에 따라 텍스트를 분할하기 때문에 연관된 내용이 분리되어 문맥이 끊길 수 있는 문제가 있습니다.
- 의미 기반 분할은 텍스트의 의미적 연관성을 고려햐여 일관된 청크를 생성하는데 효과적이지만, 긴 문서나 복잡한 주제를 다룰 때 문서의 전체적인 구조와 계층적 관계를 완벽하게 포착하기에는 한계가 있습니다.

[ch04_PARENT_CHILD_CHUNKING.ipynb](ch04_PARENT_CHILD_CHUNKING.ipynb)

## 2. 질의 변형
- 사용자의 원래 질문을 보다 효과적인 검색 쿼리로 변환하는 과정을 말합니다.
- 사용자의 질문이 모호하거나 검색에 최적화되지 않은 형태인 경우, 연관 문서를 제대로 검색할수 없어 RAG 시스템의 정확도가 떨어지게 됩니다.
- 질의 변형은 원래의 질문을 분석하고 재구성하여 이러한 문제를 해결합니다.

### 2.1 다중 질의 생성
- 사용자의 원래 질문을 바탕으로 여러 개의 다양한 쿼리를 생성하는 방법입니다.
- LLM을 사용하여 원래 질문을 다양한 관점으로 변형시킨 여러 개의 질문을 생성합니다.
- 생성된 각 쿼리를 사용하여 독립적으로 문서를 검색합니다.(병렬검색)
- 검색 결과를 종합하여 중복을 제거하고 가장 관련성 높은 문서들을 선별합니다.

##### 예시
- 주식 투자를 처믕 시작하려면 어떻게 해야 하나요?
- 초보 투자자를 위한 주식 투자 기초 지식은 무엇인가?
- 주식 시장 분석을 위한 기본적인 재무제표 읽는 법은?
- 주식 투자 시작 전 필요한 자금 관리 전략은?

- 보다 다양한 측면에서 정보를 포괄적으로 검색하여 다양한 문서를 수집하게 합니다.
- 하나의 질문을 여러 가지 방식으로 해석하고 쿼리를 생성함으로써, 사용자의 의도를 더 정확히 파악하고 그에 맞는 결과를 제공합니다.

[ch04_MULTIQUERY_GENERATION.ipynb](ch04_MULTIQUERY_GENERATION.ipynb)

## 2.2 가상 문서 임베딩(HyDE)
- 기존 쿼리에 대해 가상의 문서를 만들어서 이를 기존 쿼리 대신 활용하는 방법입니다.
- 질문을 이용하는 것이 아닌, 질문에 대한 답변을 기반으로 문서를 찾는 방식입니다.
- 검색 대상 문서가 질문보다 답변과 더 높은 의미적 연관성을 가지는 경우가 많다는 점을 활용한 접근입니다.

- 예시
- 금융 서적을 대상으로 한 RAG 시스템을 예로 생각해 봅시다.
- 편의상 이 시스템의 유사도 검색은 키워드를 활용한 bm25 방식을 활용한다고 가정합니다.
- "주식시장의 변동성이 높을 때 투자 전략은 무엇인가요?"와 같은 질문을 그대로 검색에 활용한다고 가정할때 사용할 수 있는 키워드는 다음과 같습니다.
  - 주식시장, 변동성, 투자 전략
- 이 질문에 대한 답변인 "주식 시장의 변동성이 높을 때는 분산 투자, 달러 코스트 애퍼리징, 안전 자산 비중 확대 등의 전략을 고려할 수 있습니다."
  - 분산 투자, 달러 코스트 애버리징, 안전 자산, 비중 확대
- 답변을 활용하는 방식으로 문서 검색 시 더 정확하고 관련성 높은 문서를 찾을 가능성을 크게 높입니다.
- 단점은 가상의 답변을 생성하기 위한 LLM을 사용하기 때문에 계산 비용이 발생합니다.
- 검색 지연시간이 증가할수 있어 빠른 응답이 요구되는 애플리케이션에서는 제약이 될 수 있습니다.

[ch04_HYDE.ipynb](ch04_HYDE.ipynb)